In [1]:
#Validation for case Disposition through A2I

In [2]:
#import the libraries
import sagemaker
import boto3
import pickle,os
from bs4 import BeautifulSoup
import pyarrow.parquet as pq
import numpy as np
import pandas as pd
import io
import re
import pprint
import uuid
import time
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sagemaker import get_execution_role
from s3fs.core import S3FileSystem
from bs4 import BeautifulSoup
import boto3
from botocore.exceptions import ClientError

In [3]:
#INPUT BUCKET
BUCKET = "wk-poc"
PREFIX_FIRST = "Validation_A2I/"
BUCKET_output = 'wk-poc'
OUTPUT_PATH = f's3://{BUCKET_output}/a2i-results'
REGION_NAME ='us-east-1'
TASK_TIME =60
NEW_PREFIX = 'Archival/'
s3 = boto3.resource('s3')
OLD_BUCKET = s3.Bucket(BUCKET)
NEW_BUCKET= s3.Bucket(BUCKET)
NUM_OF_BATCH =3


In [4]:
SENDER = "saurabh.paal@outlook.com"
RECIPIENT_LIST = ["Sourabh.Pal@wolterskluwer.com"]
CLIENT_NOTIFICATION = boto3.client('ses',region_name=REGION_NAME)
TIMESTAMP = time.strftime("%Y-%m-%d-%H-%M", time.gmtime())
TIMESTAMP_EMAIL_SENDER = time.strftime("%Y-%m-%d", time.gmtime())

In [5]:
REGION = boto3.session.Session().region_name
session = boto3.session.Session(region_name= REGION_NAME)
client = session.client('s3')

In [6]:
role=sagemaker.get_execution_role()  
s3_file = S3FileSystem()
get_folder_objects = client.list_objects_v2(
        Bucket = BUCKET,
        Prefix = PREFIX_FIRST
      )

In [7]:
# EXTRACTING THE DATA FROM INPUT BUCKET
filenames = []
List_of_files = []

for file_name in get_folder_objects['Contents']:    
    file_key=file_name['Key']
    
    filenames.append(file_key)
    #print(file_key)
    data_location = 's3://{}/{}'.format(BUCKET, file_key)
    List_of_files.append(data_location)

del (filenames[0])
del (List_of_files[0])





In [8]:
List_of_files

['s3://wk-poc/Validation_A2I/100.parquet',
 's3://wk-poc/Validation_A2I/101.parquet',
 's3://wk-poc/Validation_A2I/102.parquet',
 's3://wk-poc/Validation_A2I/103.parquet',
 's3://wk-poc/Validation_A2I/104.parquet',
 's3://wk-poc/Validation_A2I/105.parquet',
 's3://wk-poc/Validation_A2I/106.parquet',
 's3://wk-poc/Validation_A2I/107.parquet',
 's3://wk-poc/Validation_A2I/108.parquet',
 's3://wk-poc/Validation_A2I/109.parquet',
 's3://wk-poc/Validation_A2I/110.parquet',
 's3://wk-poc/Validation_A2I/111.parquet']

In [9]:
def Batching_for_dataframes(number_of_batch):
    lst=List_of_files
    temp_list = []
    df_batch_list = []
    for file in lst:
        temp_list.append(pd.read_parquet(file).astype(str))
    df = pd.concat(temp_list).reset_index(drop=True)
    df_batch_list = np.array_split(df, number_of_batch)
        
    return df_batch_list

In [10]:

def Batching_for_dataframes(number_of_batch):
    lst=List_of_files
    
    batch = [lst[i:i + number_of_batch] for i in range(0, len(lst), number_of_batch)]
    
    list_of_files_ = batch[:1][0]
    dfs = {i:pd.read_parquet(file) for i,file in enumerate(list_of_files_)}
    main_df = pd.concat(dfs.values())
    main_df['index'] = range(len(main_df))
    main_df.set_index('index',inplace=True)

    range_of_data=len(main_df)

    for file in batch:
        print('file read',file)
        for file_key in file:
            print(file_key)
#         copy files from old folder to new folder
            obj_key = file_key[12:]
            print("obj_key",obj_key)

            old_source = { 'Bucket': BUCKET,
                               'Key': obj_key}
            
            new_key = obj_key.replace(PREFIX_FIRST, NEW_PREFIX, 1)
            print(new_key)
            new_obj = NEW_BUCKET.Object(new_key)
            new_obj.copy(old_source)
            #print('new key: {} \n old key: {}'.format(obj_key,new_key))
#         delete files from old folder
#             obj = s3.Object(BUCKET, obj_key)
#             obj.delete()
            #print('\n\nobject with key {} deleted from {}'.format(obj_key,old_bucket_name))
        return main_df,range_of_data
   

In [11]:
# lst=List_of_files
    
# batch = [lst[i:i + 2] for i in range(0, len(lst), 2)]

# list_of_files_ = batch[:1][0]
# dfs = {i:pd.read_parquet(file) for i,file in enumerate(list_of_files_)}

# main_df = pd.concat(dfs.values())
# main_df['index'] = range(len(main_df))
# main_df.set_index('index',inplace=True)
# main_df["summary"]=main_df["summary"][1][1000:]
# range_of_data=len(main_df)

In [12]:
main_df,range_of_data = Batching_for_dataframes(NUM_OF_BATCH)

file read ['s3://wk-poc/Validation_A2I/100.parquet', 's3://wk-poc/Validation_A2I/101.parquet', 's3://wk-poc/Validation_A2I/102.parquet']
s3://wk-poc/Validation_A2I/100.parquet
obj_key Validation_A2I/100.parquet
Archival/100.parquet
s3://wk-poc/Validation_A2I/101.parquet
obj_key Validation_A2I/101.parquet
Archival/101.parquet
s3://wk-poc/Validation_A2I/102.parquet
obj_key Validation_A2I/102.parquet
Archival/102.parquet


In [13]:
main_df

,generator,collection,nbccite,party_name,court_date,docket_no,case_date,case_date1,court,court1,...,series0,page0,cite_check_count,all_page_nos,appeal_info,volume1,series1,page1,disposition_keywords,prediction_probability
index,,,,,,,,,,,,,,,,,,,,,
0,"HTML Tidy for Windows (vers 14 October 2008), ...",FED01CASEF3D,<NA>,"Hayat SINDI, Plaintiff, Appellee, v. Samia EL-...",<NA>,16-2347,20180711,2018/07/11,"United States Court of Appeals, First Circuit","United States Court of Appeals, First Circuit",...,F.3d,1,<NA>,"896 F.3d 1, 896 F.3d 2, 896 F.3d 3, 896 F.3d 4...",APPEAL FROM THE UNITED STATES DISTRICT COURT F...,<NA>,<NA>,<NA>,vacated,61.210812
1,"HTML Tidy for Windows (vers 14 October 2008), ...",FED02CASEF3D,<NA>,"ANDERSON NEWS, L.L.C., Plaintiff-Counter-Defen...",<NA>,"15-2714-cv(L), 15-2889-cv(XAP), 15-2894-cv(XAP...",20180719,2018/07/19,"United States Court of Appeals, Second Circuit","United States Court of Appeals, Second Circuit",...,F.3d,87,<NA>,"899 F.3d 87, 899 F.3d 88, 899 F.3d 89, 899 F.3...",<NA>,<NA>,<NA>,<NA>,reversed,26.029163
2,"HTML Tidy for Windows (vers 14 October 2008), ...",FEDCASEUSPQ,<NA>,IN RE STEPHEN W. COMISKEY,<NA>,2006-1286,20090113,2009/01/13,"United States Court of Appeals, Federal Circuit","United States Court of Appeals, Federal Circuit",...,U.S.P.Q.2d,1641,<NA>,"89 U.S.P.Q.2d 1641, 89 U.S.P.Q.2d 1642, 89 U.S...",<NA>,<NA>,<NA>,<NA>,remanded,26.284885


In [14]:
#Private workflow ARN
WORKTEAM_ARN= "arn:aws:sagemaker:us-east-1:964662317001:workteam/private-crowd/case-validation"


In [15]:
# Amazon SageMaker client
sagemaker_client = boto3.client('sagemaker', REGION)

# Amazon Augment AI (A2I) client
a2i = boto3.client('sagemaker-a2i-runtime')

# Amazon S3 client 
s3 = boto3.client('s3', REGION)

flowDefinitionName = 'wk-validation-case-deposition-demo-' +TIMESTAMP

# Task UI name - this value is unique per account and REGION. You can also provide your own value here.
taskUIName = 'ui-sagemaker-tabular-data-' + TIMESTAMP 

In [16]:
#create the UI template for a2i
template = r"""
<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>

<style>
  table, tr, th, td {
    border: 1px solid black;
    border-collapse: collapse;
    padding: 5px;
  }
</style>
<script> 
  function enable(opt_dis,index){
    var btn_val = document.getElementById("label"+ index);
    btn_val.disabled = opt_dis.checked ? false: true;
    if(!btn_val.disabled){
      btn_val.focus();
    }
  }
  function disable(opt_agr,index){
    var btn_val = document.getElementById("label"+ index);
    btn_val.disabled = opt_dis.checked ? false: true;
    if(!btn_val.disabled){
      btn_val.focus();
    }
  }

</script>  

<crowd-form>
    <h1 class="center">Validation For Case Deposition</h1>
    <style> 
            body { 
                text-align:center; 
            } 
            h1 { 
                color:black; 
            } 
    </style>
    <div>
        <h1>Instructions</h1>
        <p>Please review the predictions in the Predictions table based on the input data table below, and make corrections where appropriate. </p>
        
    </div>
    
    <br>
    <h1> Predictions Table </h1>
    <style> 
            body { 
                text-align:center; 
            } 
            h1 { 
                color:black; 
            } 
    </style>
    <table>
      <tr>
        <th>ROW NUMBER</th> 
        <th>Case_ID</th>
        <th>Summary</th>
        <th>MODEL PREDICTION</th>        
        <th>AGREE/DISAGREE WITH ML RATING?</th>
        <th>YOUR PREDICTION</th>
        <th>CHANGE REASON </th>
      </tr>

      {% for pair in task.input.Pairs %}

           
          <tr>
          <td>{{ pair.row }}</td>
          <td><crowd-text-area name="case_id{{ forloop.index }}" value="{{ pair.case_id }}"></crowd-text-area>
          <td><crowd-text-area name="text{{ forloop.index }}" value="{{ pair.text }}"></crowd-text-area>          
          <td><crowd-text-area name="predicted{{ forloop.index }}" value="{{ pair.prediction }}"></crowd-text-area></td>
          
          <td>
            <p>
              <input type="radio" id="opt_agr" name="rating{{ forloop.index }}" value="agree" onclick="disable(this,{{ forloop.index }})" >
              <label for="agree{{ forloop.index }}">Agree</label>
            </p>
            <p>
              <input type="radio" id="opt_dis" name="rating{{ forloop.index }}" value="disagree" onclick="enable(this,{{ forloop.index }})">
              <label for="disagree{{ forloop.index }}">Disagree</label>       
             </td>
         </p>
         <td>
            
             <select name=Human_answer{{ forloop.index }} id="label{{ forloop.index }}" disabled="disabled" required>
                <option value="">Choose a Label</option>
                <option value="Affirmed" >Affirmed</option>  
                <option value="Reversed">Reversed</option>
                <option value="Denied">Denied</option>
                <option value="Remanded">Remanded</option>
                <option value="Vacated">Vacated</option>
                <option value="Dismised">Dismised</option>
                <option value="Rejected">Rejcted</option>
                <option value="Granted">Granted</option>
                <option value="Amended">Amended</option>
                <option value="Afirmed in part">Afirmed in part</option>     
                
                
            
          </td>
          <td>
            <p>
            <input type="text" name="Change Reason{{ forloop.index }}" placeholder="Explain why you changed the prediction"/>
            </p>
           </td>
          
        </tr>

      {% endfor %}

    </table>
   
   
  
</crowd-form>



"""

#create the response for a2i 
def create_task_ui():
    '''
    Creates a Human Task UI resource.

    Returns:
    struct: HumanTaskUiArn
    '''
    response = sagemaker_client.create_human_task_ui(
        HumanTaskUiName=taskUIName,
        UiTemplate={'Content': template})
    return response

In [17]:
# Create task UI
humanTaskUiResponse = create_task_ui()
humanTaskUiArn = humanTaskUiResponse['HumanTaskUiArn']
print(humanTaskUiArn)

arn:aws:sagemaker:us-east-1:964662317001:human-task-ui/ui-sagemaker-tabular-data-2021-02-22-14-43


In [18]:
##creating flow definition
create_workflow_definition_response = sagemaker_client.create_flow_definition(
        FlowDefinitionName= flowDefinitionName,
        RoleArn= role,
        HumanLoopConfig= {
            "WorkteamArn": WORKTEAM_ARN,
            "HumanTaskUiArn": humanTaskUiArn,
            "TaskCount": 1,
            "TaskDescription": "Make sure the disposition_keywords are correct",
            "TaskTitle": "AFJAA-validation-for-case-deposition" + TIMESTAMP
        },
        OutputConfig={
            "S3OutputPath" : OUTPUT_PATH
        }
    )
flowDefinitionArn = create_workflow_definition_response['FlowDefinitionArn']

In [19]:
create_workflow_definition_response

{'FlowDefinitionArn': 'arn:aws:sagemaker:us-east-1:964662317001:flow-definition/wk-validation-case-deposition-demo-2021-02-22-14-43',
 'ResponseMetadata': {'RequestId': 'fc4247eb-f55b-4570-a4de-ae0aaed8e9fe',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fc4247eb-f55b-4570-a4de-ae0aaed8e9fe',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '132',
   'date': 'Mon, 22 Feb 2021 14:43:24 GMT'},
  'RetryAttempts': 0}}

In [20]:
# Make the flow definition an intialize the same
for x in range(TASK_TIME):
    describeFlowDefinitionResponse = sagemaker_client.describe_flow_definition(FlowDefinitionName=flowDefinitionName)
    print(describeFlowDefinitionResponse['FlowDefinitionStatus'])
    if (describeFlowDefinitionResponse['FlowDefinitionStatus'] == 'Active'):
        print("Flow Definition is active")
        break
    time.sleep(2)

Initializing
Active
Flow Definition is active


In [21]:
#selecting the ItemList for UI
main_df = main_df[['uid','short_summary','disposition_keywords']].reset_index(drop=True)
item_list = [{'row': "{}".format(x),"text":main_df.short_summary[x],"case_id": main_df.uid[x], 'prediction': main_df.disposition_keywords[x]} for x in range(range_of_data)]
ip_content = {"table": main_df.head(range_of_data).to_html(), 
          'Pairs': item_list
         }


KeyError: "['disposition_keyword'] not in index"

In [ ]:
import json
humanLoopName = str(uuid.uuid4())

start_loop_response = a2i.start_human_loop(
            HumanLoopName=humanLoopName,
            FlowDefinitionArn=flowDefinitionArn,
            HumanLoopInput={
                "InputContent": json.dumps(ip_content)
            }
        )
start_loop_response


In [ ]:
start_loop_response

In [ ]:
##checking status for human loop
 
resp = a2i.describe_human_loop(HumanLoopName=humanLoopName)
print(f'HumanLoop Name: {humanLoopName}')
print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
print('\n')
    
if resp["HumanLoopStatus"] == "Completed":
    completed_human_loops.append(resp)

In [ ]:
resp

In [ ]:
workteamName = WORKTEAM_ARN[WORKTEAM_ARN.rfind('/') + 1:]
print("Navigate to the private worker portal and do the tasks. Make sure you've invited yourself to your workteam!")
print('https://' + sagemaker_client.describe_workteam(WorkteamName=workteamName)['Workteam']['SubDomain'])

In [ ]:
# Try to send the email.https://b7ncf4dyaa.labeling.us-east-1.sagemaker.aws/
# The email body for recipients with non-HTML email clients.
def email_sent():
    
    SUBJECT = "AFJAA || Case Disposition Validation || "  +TIMESTAMP_EMAIL_SENDER
    BODY_TEXT = ("Hi ,\n\n"
                 "A task is created and it is waiting for response.The below are the task details. \n\n\n"
                 "Task Title : AFJAA-Validation-For-Case-Disposition. \n" 
                 "Task Description : Make sure the disposition_keywords are correct  \n"
                 "Link to access the document: "+'https://' + sagemaker_client.describe_workteam(WorkteamName=workteamName)['Workteam']['SubDomain']+"\n\n\n"
                 "if the above URL is not working please reach out to Support/DEV team"

                )        

    # The character encoding for the email.
    CHARSET = "UTF-8"

    try:
        #Provide the contents of the email.
        response = CLIENT_NOTIFICATION.send_email(
            Destination={
                'ToAddresses': [RECIPIENT for RECIPIENT in RECIPIENT_LIST],
            },
            Message={
                'Body': {

                    'Text': {
                        'Charset': CHARSET,
                        'Data': BODY_TEXT,
                    },
                },
                'Subject': {
                    'Charset': CHARSET,
                    'Data': SUBJECT,
                },
            },
            Source=SENDER,


        )

    except ClientError as e:
        return(e.response['Error']['Message'])
    else:
        return("Email sent! Message ID:")+ response['MessageId']
        

In [ ]:
email_sent()

In [ ]:
#  <div style="text-align:center">  
#     <crowd-button style="margin:25px;" form-action="submit" variant="primary" data-testid="crowd-submit">SUBMIT</crowd-button>
#     </div>

In [ ]:
#TODO--> DUMPING INTO THE BUCKET
#MERGE THE OUTPUT TO CURATED BUCKET